<a href="https://colab.research.google.com/github/raghumriyer/colab/blob/master/Generate_story.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load Larger LSTM network and generate text
import sys
import os
import collections
import string
import numpy
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


In [137]:
# load ascii text and covert to lowercase
from google.colab import drive
drive.mount('/content/drive')
file = "wonderland.txt"
filename = os.path.abspath("/content/drive/My Drive/EIP3/"+file)

#filename = "wonderland.txt"
raw_text1 = open(filename).read()
raw_text1 = raw_text1.lower()

table = str.maketrans({key: None for key in string.punctuation})
raw_text = raw_text1.translate(table) 

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total Characters:  154861
Total Vocab:  39


In [138]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
dataX = pad_sequences(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights

file = "weights-improvement-09-1.5390-bigger.hdf5"
#file = "weights-improvement-63-1.0572.hdf5"
filename = os.path.abspath("/content/drive/My Drive/EIP3/"+file)
#filename = "weights-improvement-09-1.5390-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

print("The story:\n")
# generate characters
print("\"")
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	a = numpy.array([index])
	pattern=numpy.append(pattern,a)
	pattern = pattern[1:len(pattern)]
  
print("\"")
print("\nDone.")


Total Patterns:  154761
Seed:
" g the eggs said the pigeon
but i must be on the lookout for serpents night and day why i
havent had  "
The story:

"
the minu of the sabbit she was not and she would gall the ming and the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turtle said the mock turt"

Done.
